In [1]:
import os
import pyspark
from haversine import haversine, Unit
from pyspark.sql.functions import avg, col, desc, hour
from pyspark.sql.functions import udf,lit, cast, col
from pyspark.sql.types import DoubleType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf,lit, cast, col, asc, desc
from pyspark.sql.window import Window
from pyspark.sql.functions import expr
import re
from pyspark.sql.functions import lag
from pyspark.sql.types import FloatType
from pyspark.sql.functions import col, dense_rank, desc, asc
from pyspark.sql.functions import regexp_replace, regexp_extract, lower, explode, split, col, desc

conf = pyspark.SparkConf()
conf.set('spark.ui.proxyBase', '/user/' + os.environ['JUPYTERHUB_USER'] + '/proxy/4041')
conf.set('spark.sql.repl.eagerEval.enabled', True)
conf.set('spark.driver.memory','4g')

sc = pyspark.SparkContext(conf=conf)
spark = pyspark.SQLContext.getOrCreate(sc)



Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/25 22:58:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


/opt/conda/envs/bigdata-fall22/lib/python3.7/site-packages/pyspark/sql/context.py:159: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


## 2

In [2]:
bigram = sc.textFile("shared/hw2/romeo-juliet-pg1777.txt")\
             .map(lambda line: re.sub(r'[^a-zA-Z0-9]',' ', line))\
             .map(lambda line: " ".join(line.lower().split()))\
             .map(lambda line: line.lower().split(' '))\
             .filter(lambda line : not(len(line)==1))\
             .flatMap(lambda xs: (x for x in zip(xs, xs[1: ])))\
             .map(lambda each_word: (each_word, 1)) \
             .reduceByKey(lambda i, j: i+j)\
             .sortBy(lambda x: x[1], False)\
            
             
bigram_list = bigram.collect()

In [3]:
bigram_list = bigram.collect()[0:5]

In [4]:
unigram = sc.textFile("shared/hw2/romeo-juliet-pg1777.txt")\
            .flatMap(lambda line:(lambda line: re.sub(r'[^a-zA-Z0-9]',' ', line))(line).split())\
            .map(lambda each_word: (each_word.lower(), 1)) \
            .reduceByKey(lambda i, j: i+j)\
            .sortBy(lambda x: x[1], False)
unigram

PythonRDD[25] at RDD at PythonRDD.scala:53

In [5]:
unigram_list = unigram.collect()[0:5]
print(unigram_list[0][1])

780


Method 1

In [6]:
for i in bigram_list:
    for j in unigram_list:
        if j[0] == i[0][0]:
            print(i[0]," and ",j[0]," = ",int(i[1])/int(j[1]))



('i', 'll')  and  i  =  0.1094224924012158
('i', 'will')  and  i  =  0.0911854103343465
('i', 'am')  and  i  =  0.0790273556231003


Method 2

In [7]:
bigram_df=spark.createDataFrame(bigram_list,['bigram', 'bigrams_count'])
unigram_df=spark.createDataFrame(unigram_list,['unigram', 'unigrams_count'])

In [8]:
bigram_df = bigram_df.withColumn('bigram_string', bigram_df.bigram.cast("string"))
bigram_df = bigram_df.withColumn('bigram_string', regexp_replace(col("bigram_string"),'[^0-9a-z]',' '))
split_col = pyspark.sql.functions.split(bigram_df['bigram_string'],' ')
bigram_df = bigram_df.withColumn('bigram_Word_1', split_col.getItem(1)) \
               .withColumn('bigram_Word_2', split_col.getItem(3)) \



In [9]:
bigram_df

+---------+-------------+-------------+-------------+-------------+
|   bigram|bigrams_count|bigram_string|bigram_Word_1|bigram_Word_2|
+---------+-------------+-------------+-------------+-------------+
|  {i, ll}|           72|       i  ll |            i|           ll|
|{of, the}|           63|     of  the |           of|          the|
|{i, will}|           60|     i  will |            i|         will|
|  {i, am}|           52|       i  am |            i|           am|
|{in, the}|           46|     in  the |           in|          the|
+---------+-------------+-------------+-------------+-------------+

In [10]:
unigram_df

+-------+--------------+
|unigram|unigrams_count|
+-------+--------------+
|    and|           780|
|    the|           749|
|      i|           658|
|     to|           616|
|      a|           485|
+-------+--------------+

In [11]:
unigram_bigram = unigram_df.join(bigram_df,unigram_df.unigram ==  bigram_df.bigram_Word_1,"inner")


In [12]:
unigram_bigram.withColumn("Probability",col("bigrams_count")/col("unigrams_count"))

+-------+--------------+---------+-------------+-------------+-------------+-------------+------------------+
|unigram|unigrams_count|   bigram|bigrams_count|bigram_string|bigram_Word_1|bigram_Word_2|       Probability|
+-------+--------------+---------+-------------+-------------+-------------+-------------+------------------+
|      i|           658|  {i, ll}|           72|       i  ll |            i|           ll|0.1094224924012158|
|      i|           658|{i, will}|           60|     i  will |            i|         will|0.0911854103343465|
|      i|           658|  {i, am}|           52|       i  am |            i|           am|0.0790273556231003|
+-------+--------------+---------+-------------+-------------+-------------+-------------+------------------+

## 3

In [13]:
import pandas as pd
from pyspark.sql.types import StringType
from pyspark.sql.types import BooleanType
import glob
from bloom_filter2 import BloomFilter #probabilistic data structure
from pyspark.sql.functions import broadcast

In [14]:
#Broadcast filter cannot do for df because df is stored as graph

In [15]:
exclude = ["shared/midterm/drive_stats_2019_Q1/2019-03-31.csv"]
csv_files = []

for files in glob.glob("shared/midterm/drive_stats_2019_Q1/*.csv"):
    if files not in exclude:
        csv_files.append(files)

df_A = spark.read.option('delimiter', ',')\
                 .option('header', True)\
                 .option('inferSchema', True)\
                 .csv(csv_files)

In [16]:
df_B = spark.read.option("inferSchema", "true").option("header", "true").csv("shared/midterm/drive_stats_2019_Q1/2019-03-31.csv")
unique_model_B = list(df_B.select(col("model")).distinct().collect())

In [17]:
bloom = BloomFilter(max_elements = len(unique_model_B), error_rate = 0.001)
for i in unique_model_B:
    if i[0] not in bloom:
        bloom.add(i[0])

In [18]:
def bloom_one_row(a):
    bloom_filter_local = bv.value
    if a in bloom_filter_local:
        return True
    else:
        return False
        
bloom_udf = udf(bloom_one_row, BooleanType())
bv = sc.broadcast(bloom)

In [19]:
df_A = df_A.withColumn("In/Out_Bloom", bloom_udf(col("model")))
df_A.groupby(col("In/Out_Bloom")).count().orderBy('count')

+------------+-------+
|In/Out_Bloom|  count|
+------------+-------+
|       false|    659|
|        true|9470149|
+------------+-------+

#### Method 1

In [28]:
df_A_model = df_A[["model"]].limit(100)
df_B_model = df_B[["model"]].limit(100)
df_A_join = df_A_model.join(broadcast(df_B_model), on='model', how='inner')

AttributeError: 'DataFrame' object has no attribute 'astype'

In [ ]:
df_bloompresent = df_A.filter(col("In/Out_Bloom") == lit(True)).count()

In [ ]:
df_bloompresent[~df_A_join.isin(df_bloompresent)]

#### Method 2

In [ ]:
df_A_join = df_A.join(broadcast(df_B), on='model', how='leftanti')
df_A_join.count()

## 4

In [22]:
from pyspark.sql.functions import substring
from pyspark.sql.functions import col, dense_rank, desc
from pyspark.sql.window import Window

In [23]:
df = spark.read.option('delimiter', ',')\
                .option('header', True)\
                .option('inferSchema', True)\
                .csv('shared/hw2/BreadBasket_DMS.csv')


In [24]:
df_hour = df.withColumn("hour", substring(col("Time").cast("String"), 12,2)).sort(col("hour")).groupby(col("Date"), col("hour"), col("Item")).count().sort(col("hour"), col("count")).filter(col("Item") != "NONE")


In [25]:
min_window = Window.partitionBy(col("Date"),col("hour")).orderBy(col("count"), col("Item"))
bottom_two = df_hour.withColumn("bottom_2", dense_rank().over(min_window)).where(col("bottom_2") <=2)
bottom_two = bottom_two.withColumn("Type",lit("min"))

In [26]:
max_window = Window.partitionBy(col("Date"),col("hour")).orderBy(desc(col("count")), col("Item"))
top_two = df_hour.withColumn("top_2", dense_rank().over(max_window)).where(col("top_2") <= 2) 
top_two = top_two.withColumn("Type",lit("max"))

In [ ]:
total = top_two.union(bottom_two)
total.orderBy(asc(col("Date")), asc(col("hour"), asc(col("Type")))

## 1

In [ ]:
Top k in mapper then reservoir sampling in reducer

In [ ]:
Top k in mapper and top k in reducer